In [1]:
from keras.preprocessing.image import ImageDataGenerator, load_img

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K

import os

import numpy as np

import pandas as np

import matplotlib.pyplot as plt

%matplotlib inline


Using TensorFlow backend.


In [2]:
img_width, img_height = 150, 150

In [3]:

train ='C:/Users/mysh7/OneDrive/Desktop/ai stanley/chest-xray-pneumonia/chest_xray/chest_xray/train'

val = 'C:/Users/mysh7/OneDrive/Desktop/ai stanley/chest-xray-pneumonia/chest_xray/chest_xray/val'

test = 'C:/Users/mysh7/OneDrive/Desktop/ai stanley/chest-xray-pneumonia/chest_xray/chest_xray/test'

In [4]:

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [6]:
#creating model 
model = Sequential()
#adding layers to the model 
#32 filters:the dimensionality of the output space (i.e. the number of output filters in the convolution).
#(3,3) kernel_size: An array specifying the dimension of the 2D convolution window.
model.add(Conv2D(32, (3, 3), input_shape=input_shape))

#Activation function : (softmax,elu,selu,softplus,softsign,relu,tanh,sigmoid,hard_sigmoid),
#the activation function of a node defines the output of that node given an input or set of inputs.
model.add(Activation('relu'))
#maxpooling2d
model.add(MaxPooling2D(pool_size=(2, 2)))




model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))



model.add(Flatten())

model.add(Dense(50))

model.add(Activation('relu'))

model.add(Dropout(50))

model.add(Dense(1))

model.add(Activation('sigmoid'))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_4 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [8]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [9]:
#ImageDataGener
train_datagen = ImageDataGenerator(rescale=1. / 255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [10]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [12]:
validation_generator = test_datagen.flow_from_directory(
    val,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

Found 16 images belonging to 2 classes.


In [13]:
test_generator = test_datagen.flow_from_directory(
    test,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

Found 624 images belonging to 2 classes.


In [14]:
model.fit_generator(
    train_generator,
    steps_per_epoch=5217 // 16,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=17 // 16)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
326/326 [==============================] - 118s 363ms/step - loss: 0.4317 - acc: 0.8207 - val_loss: 0.3261 - val_acc: 0.8125
Epoch 2/20
326/326 [==============================] - 104s 319ms/step - loss: 0.2619 - acc: 0.8961 - val_loss: 2.0861 - val_acc: 0.6250
Epoch 3/20
326/326 [==============================] - 106s 325ms/step - loss: 0.2100 - acc: 0.9214 - val_loss: 0.6142 - val_acc: 0.6250
Epoch 4/20
326/326 [==============================] - 104s 319ms/step - loss: 0.1896 - acc: 0.9300 - val_loss: 1.5634 - val_acc: 0.6250
Epoch 5/20
326/326 [==============================] - 98s 299ms/step - loss: 0.1703 - acc: 0.9346 - val_loss: 0.6629 - val_acc: 0.6875
Epoch 6/20
326/326 [==============================] - 98s 301ms/step - loss: 0.1638 - acc: 0.9427 - val_loss: 2.9174 - val_acc: 0.5625
Epoch 7/20
326/326 [==============================] - 101s 311ms/step - loss: 0.1681 - acc: 0.9398 - val_loss: 1.1629 - val_acc: 0.6250
Ep

In [15]:
model.save('v1.h5')

In [16]:
scores = model.evaluate_generator(test_generator,624/16,workers=74)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


acc: 92.31%


In [1]:
from keras.models import model_from_json

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

json_file = open('C:/Users/mysh7/OneDrive/Desktop/ai stanley/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model1.h5")


loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

scores = model.evaluate_generator(test_generator,624/16,workers=30)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Using TensorFlow backend.


NameError: name 'model' is not defined